# Install and import necessary modules

In [ ]:
!pip install simpletransformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 114.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 26.1 MB/s eta 0:00:

In [ ]:
import random
import pickle
import logging
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs

## Connection to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Telecom/Airbus/Intimacy"

## Set seed and logger

In [ ]:
seed = 56
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Load data

In [ ]:
# Read train data
df = pd.read_csv("train.csv")
df = df.sample(frac=1, random_state=0)
train_texts = list(df.text.values)
train_labels = list(df.label.values)

# Read translated train data
with open('translated_train.pickle', 'rb') as f:
    tr_train_texts = pickle.load(f)

# Pre-processing

In [ ]:
# Combine original and translated train texts
train_texts = [train_texts[i] + ' </s></s> ' + tr_train_texts[i] for i in range(len(train_texts))]

# Create train_df from combined train texts and labels
train_df = pd.DataFrame({"text": train_texts, "labels": train_labels})

# Split train_df into train, val, and test sets
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=0)

# Initialize model

In [ ]:
epochs = 4
max_seq_length = 162
use_cuda = True

# Define model arguments
model_args = ClassificationArgs(
    num_train_epochs=epochs, overwrite_output_dir=True,
    no_save=False, max_seq_length=max_seq_length, regression=True
)

# Initialize the model
model = ClassificationModel(
    "xlmroberta", "cardiffnlp/twitter-xlm-roberta-base", args=model_args, use_cuda=use_cuda, num_labels=1
)

# other possibilities
#"xlmroberta", "xlm-roberta-base", args=model_args, use_cuda=True, num_labels=1
#"bert", "bert-base-multilingual-cased", args=model_args, use_cuda=True, num_labels=1

# Train and test the model

In [ ]:
model.train_model(train_df)

# Evaluate on the test set
test_texts = list(test_df["text"].values)
test_labels = list(test_df["labels"].values)

predictions, _ = model.predict(test_texts)
r2 = r2_score(predictions, test_labels)
print(f"Test R2 Score: {r2}")


In [ ]:
#access the hugging face model for saving
model.model.save_pretrained('model_intimacy')
model.tokenizer.save_pretrained('model_intimacy')
model.config.save_pretrained('model_intimacy/')